<a href="https://colab.research.google.com/github/vinx-2105/CIFAR-10_Basic_Classifier/blob/master/CIFAR-10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

tcmalloc: large alloc 1073750016 bytes == 0x58128000 @  0x7fc2308302a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [1]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

NameError: ignored

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms

In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
root_path = '/gdrive/'

In [0]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [7]:
#transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(((0.5,0.5,0.5),(0.5,0.5,0.5)))])
trainset = torchvision.datasets.CIFAR10(root='./gdrive/pytorch/train_data', train=True, download=True, transform=transform)

In [8]:
testset = torchvision.datasets.CIFAR10(root='./gdrive/pytorch/test_data', train=False, download=True, transform=transform)

In [0]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True,num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False)

In [0]:
classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

In [0]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

In [0]:
class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = torch.nn.Conv2d(3,6,5)
    self.pool = torch.nn.MaxPool2d(2,2)
    self.conv2 = torch.nn.Conv2d(6,16,5)
    self.fc1 = torch.nn.Linear(16*5*5, 120)
    self.fc2 = torch.nn.Linear(120, 84)
    self.fc3 = torch.nn.Linear(84, 10)
    
  def forward(self, x):
    x = self.pool(torch.nn.functional.relu(self.conv1(x)))
    x = self.pool(torch.nn.functional.relu(self.conv2(x)))
    x = x.view(-1,16*5*5)
    x = torch.nn.functional.relu(self.fc1(x))
    x = torch.nn.functional.relu(self.fc2(x))
    x = self.fc3(x)
    
    return x

In [0]:
net = Net()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001,momentum=0.9)

In [14]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


[1,  2000] loss: 2.187
[1,  4000] loss: 1.867
[1,  6000] loss: 1.668
[1,  8000] loss: 1.582
[1, 10000] loss: 1.496
[1, 12000] loss: 1.474
[2,  2000] loss: 1.403
[2,  4000] loss: 1.359
[2,  6000] loss: 1.332
[2,  8000] loss: 1.320
[2, 10000] loss: 1.270
[2, 12000] loss: 1.270
Finished Training


In [15]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the 10000 test images: 55 %
